In [1]:
from speed_trapv3.keypoints.model import SegmentationModel
from speed_trapv3.detection.model import RetinaNet
from speed_trapv3.detection.config import Config as DetConfig
from speed_trapv3.keypoints.config import Config as KeyConfig
from speed_trapv3.config import Config
from speed_trapv3.utils import slugify, get_prediction_path
from pathlib import Path
import random
from tqdm import tqdm
import imageio
from speed_trapv3.keypoints.dataset import crop_and_resize,process_keypoints
import numpy as np
import torchvision.transforms as T
import torch
import cv2
import os


/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
detection_model = RetinaNet().eval().cuda()
detection_model.load(DetConfig.trained_model_path)

In [3]:
keypoint_model = SegmentationModel().eval().cuda()
keypoint_model.load(KeyConfig.trained_model_path)

<All keys matched successfully>

In [4]:
image_transform = T.Compose([T.ToTensor()])

In [5]:
!rm -rf /code/data/datasets/temp_imgs/*

In [6]:
image_paths = list(Config.images_directory.glob("*.jpg"))
random.shuffle(image_paths)
score_threshold: float = 0.5
image_paths = image_paths[:3]
for image_path in tqdm(image_paths):
    slug = slugify(image_path)
    img = imageio.imread(image_path)
    img_h, img_w, _ = img.shape
    boxes = detection_model(img)
    boxes = boxes[boxes.scores > score_threshold].to_relative().to_tlbr()
    boxes = boxes.array[:,:4]
    for box in boxes:
        x1, y1, x2, y2 = (box * np.array([img_w, img_h, img_w, img_h])).astype(int)
        roi_w = x2 - x1
        roi_h = y2 - y1
        roi_resized = crop_and_resize(box, img, KeyConfig.image_crop_size[0],KeyConfig.image_crop_size[1])
        roi_resized_w, roi_resized_h = roi_resized.size
        x = image_transform(roi_resized)
        x = torch.unsqueeze(x, 0).cuda()
        keypoints = keypoint_model(x)['keypoints'][0].detach().cpu().numpy()
        keypoints =  keypoints/ np.array([roi_resized_w, roi_resized_h])#convert to relative
        keypoints = keypoints * np.array([roi_w, roi_h])
        keypoints = keypoints + np.array([x1, y1])
        img = cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), thickness=4)
        img = cv2.circle(img, (int(keypoints[0][0]), int(keypoints[0][1])), radius=5, color=(0, 0, 255), thickness=-1) #Blue: Backtire
        img = cv2.circle(img, (int(keypoints[1][0]), int(keypoints[1][1])), radius=5, color=(255, 0, 0), thickness=-1) #Red Front tire
    im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    save_path = '/code/data/datasets/temp_imgs'
    filename = str(slug) + ".jpg"
    cv2.imwrite(os.path.join(save_path, filename), im_rgb)
        

    # boxes.to_file(get_prediction_path(slug))

  0%|          | 0/3 [00:00<?, ?it/s]/tmp/ipykernel_160447/771158575.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(image_path)
/usr/local/lib/python3.9/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 3/3 [00:01<00:00,  2.77it/s]
